In [14]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn.metrics import accuracy_score, precision_score

In [15]:
df_train = pd.read_csv("fraudTrain.csv")
df_test = pd.read_csv("fraudTest.csv")



In [17]:
df_train.isnull().sum()


,0
Unnamed: 0,0
trans_date_trans_time,0
cc_num,0
merchant,0
category,0
amt,0
first,0
last,0
gender,0
street,0


In [18]:
df_test.isnull().sum()


,0
Unnamed: 0,0
trans_date_trans_time,0
cc_num,0
merchant,0
category,0
amt,0
first,0
last,0
gender,0
street,0


In [20]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()


# data preprocessing for Training Data
train_data = df_train[['gender', "category", "amt", "city", "job", "is_fraud"]]

train_data.loc[:, 'category'] = le.fit_transform(train_data["category"])
train_data.loc[:, 'city'] = le.fit_transform(train_data["city"])
train_data.loc[:, 'job'] = le.fit_transform(train_data["job"])
train_data.loc[:, 'gender'] = train_data["gender"].map({'M': 0, 'F': 1})

# data preprocessing for Testing Data
test_data = df_test[['gender',"category","amt","city", "job","is_fraud"]]

test_data.loc[:, 'category'] = le.fit_transform(test_data["category"])
test_data.loc[:, 'city'] = le.fit_transform(test_data["city"])
test_data.loc[:, 'job'] = le.fit_transform(test_data["job"])
test_data.loc[:, 'gender'] = test_data["gender"].map({'M': 0, 'F': 1})

In [21]:
train_data.head(10), test_data.head(10)


(  gender category     amt city  job  is_fraud
 0    1.0        8    4.97  500  359       0.0
 1    1.0        4  107.23  576  414       0.0
 2    0.0        0  220.11  446  297       0.0
 3    0.0        2   45.00   81  318       0.0
 4    0.0        9   41.96  205  112       0.0
 5    1.0        2   94.63  211  465       0.0
 6    1.0        3   44.54  335   27       0.0
 7    0.0        2   71.65  223  122       0.0
 8    1.0        9    4.27  452  364       0.0
 9    1.0        4  198.39  141  319       0.0,
   gender category     amt city  job  is_fraud
 0      0       10    2.86  157  275       0.0
 1      1       10   29.84   16  391       0.0
 2      1        5   41.28   61  259       0.0
 3      0        9   60.05  755  406       0.0
 4      0       13    3.19  246  196       0.0
 5      1        7   19.55   90  360       0.0
 6      1        5  133.93  117  453       0.0
 7      1       10   10.37  716  124       0.0
 8      0       12    4.37  498   13       0.0
 9      1   

fig, axes = plt.subplots(1, 2, figsize=(12, 5))

sns.set_theme()

sns.barplot(x="Age group", y="Counts", data=age_fraud, ax=axes[0])
axes[0].set_title("Number of fraudulent transactions per age group")
axes[0].set_xlabel("Age group")
axes[0].set_ylabel("Number of transactions")
axes[0].bar_label(axes[0].containers[0])

sns.barplot(x="Age group", y="Counts", data=age_not_fraud, ax=axes[1])
axes[1].set_title("Number of non-fraudulent transactions per age group")
axes[1].set_xlabel("Age group")
axes[1].set_ylabel("Number of transactions")
axes[1].bar_label(axes[1].containers[0])

In [22]:
X_train = train_data[['gender', "category", "amt", "city", "job"]]
Y_train = train_data["is_fraud"]

X_test = test_data[['gender', "category", "amt", "city", "job"]]
Y_test = test_data["is_fraud"]

In [24]:
X_train.fillna(0, inplace=True)
X_test.fillna(0, inplace=True)

<ipython-input-24-7b253b0b267d>:1: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  X_train.fillna(0, inplace=True)
<ipython-input-24-7b253b0b267d>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train.fillna(0, inplace=True)
<ipython-input-24-7b253b0b267d>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test.fillna(0, inplace=True)


In [26]:
Y_train.fillna(0, inplace=True)
Y_test.fillna(0, inplace=True)

<ipython-input-26-db7650fc96a0>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Y_train.fillna(0, inplace=True)
<ipython-input-26-db7650fc96a0>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Y_test.fillna(0, inplace=True)


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, ConfusionMatrixDisplay

In [27]:
lr_model = LogisticRegression(penalty="l1", solver="liblinear")
lr_model.fit(X_train, Y_train)

LogisticRegression(penalty='l1', solver='liblinear')

In [28]:
Y_pred = lr_model.predict(X_test)
accuracy = accuracy_score(Y_test, Y_pred)
print("Testing Accuracy Score:", accuracy)

Testing Accuracy Score: 0.99466768403588
